# Portfolio Performance vs. Benchmark

### Step 1: Import Libraries

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Visualization libraries
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image, display

# System libraries
import os
import sys
import logging
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # Font warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*inplace.*")  # Pandas warnings
logging.getLogger('matplotlib.font_manager').disabled = True

In [2]:
# UDFs
current_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'py')) 
sys.path.append(current_dir)
from quantstats_fix import *
from utils import load_and_filter_data, export_to_excel
qs.extend_pandas()

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/__init__.py
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py...
✓ Found 'plot_t

### Step 2: Define Parameters 

#### Dates

In [3]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')  # Report date
start_date = end_date - timedelta(days=5*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2020-05-29 to 2025-05-28
Time span: 1825 days (5.00 years)


#### Report File

In [4]:
output_file = f'portfolio-{datetime.date(end_date)}.xlsx'

# benchmark = 'ADME'
benchmark = pd.read_excel(output_file, sheet_name="benchmark")['Benchmark'].values[0]
print(benchmark)

DLN


#### Risk-free rate (T-bill, %)

In [5]:
# Load and process data
risk_free_df = pd.read_excel(output_file, sheet_name="daily_quotes", index_col=0)['^IRX']
# risk_free_rate  = 0.0433                              # 3‑month T‑Bill
risk_free_rate = risk_free_df.iloc[-1] / 100 

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")

Risk-Free Rate: 0.042249999999999996 -- 13 WEEK TREASURY BILL (^IRX)


### Step 3: Read Portfolio Data (Excel)

In [6]:
portfolio_df = pd.read_excel(output_file, sheet_name="equity")

# Convert percentage strings to float values
portfolio_df['Weight'] = portfolio_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_tickers = portfolio_df["Ticker"].tolist()

print(portfolio_tickers)
display(portfolio_df)

['PGR', 'DE', 'TMUS', 'XOM', 'HD', 'AMAT', 'PG', 'GILD']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,52W High,52W Low,50 Day Avg,200 Day Avg,Short Ratio,Short % of Float,Weight,Expected Return,Standard Deviation,Sharpe Ratio
0,PGR,2025-05-29,The Progressive Corporation,Financial Services,Insurance - Property & Casualty,United States,https://www.progressive.com,162120237056,166505529344,584025304,...,292.99,201.34,276.03,258.63,2.64,0.01,0.400000,0.274015,0.250518,1.093797
1,DE,2025-05-29,Deere & Company,Industrials,Farm & Heavy Construction Machinery,United States,https://www.deere.com/en,137794174976,197658083328,270509002,...,533.78,340.20,476.14,439.83,3.64,0.02,0.212173,0.241038,0.294927,0.817278
2,TMUS,2025-05-29,"T-Mobile US, Inc.",Communication Services,Telecom Services,United States,https://www.t-mobile.com,271662088192,383910346752,452736028,...,276.49,168.47,251.97,232.51,3.11,0.03,0.117479,0.165114,0.242144,0.681881
3,XOM,2025-05-29,Exxon Mobil Corporation,Energy,Oil & Gas Integrated,United States,https://corporate.exxonmobil.com,438979952640,467658211328,4300373098,...,126.34,97.80,108.39,112.68,3.05,0.01,0.070348,0.172500,0.301731,0.571702
4,HD,2025-05-29,"The Home Depot, Inc.",Consumer Cyclical,Home Improvement Retail,United States,https://www.homedepot.com,366163361792,426776199168,993296303,...,439.37,323.93,360.89,387.22,3.19,0.01,0.050000,0.078458,0.238429,0.329064
5,AMAT,2025-05-29,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,United States,https://www.appliedmaterials.com,128058613760,129598685184,799312299,...,255.89,123.74,151.36,173.73,2.58,0.02,0.050000,0.137782,0.431624,0.319218
6,PG,2025-05-29,The Procter & Gamble Company,Consumer Defensive,Household & Personal Products,United States,https://www.pginvestor.com,395641094144,418461548544,2341001776,...,180.43,156.58,164.42,168.47,2.31,0.01,0.050000,0.084647,0.172753,0.489990
7,GILD,2025-05-29,"Gilead Sciences, Inc.",Healthcare,Drug Manufacturers - General,United States,https://www.gilead.com,137423159296,151048667136,1241354188,...,119.96,62.07,105.57,94.98,2.68,0.02,0.050000,0.091269,0.232418,0.392691


### Step 4: Download Returns

In [7]:
# stock_returns = qs.utils.download_returns(ticker=portfolio_tickers, period="5y").dropna()
stock_quotes = load_and_filter_data('../data/datasets/daily_stock_quotes.csv', portfolio_tickers, start_date, end_date)
stock_returns = np.log(stock_quotes / stock_quotes.shift(1)).dropna()

benchmark_quotes = load_and_filter_data('../data/datasets/daily_benchmark_quotes.csv', benchmark, start_date, end_date)
benchmark_returns = np.log(benchmark_quotes / benchmark_quotes.shift(1)).dropna()

# Display summary statistics for all assets
print("\nSharpe Ratios for individual assets:")
sharpes = {col: qs.stats.sharpe(stock_returns[col]) for col in stock_returns.columns}
for ticker, sharpe in sharpes.items():
    print(f"{ticker}: {sharpe:.4f}")

display(stock_returns.head())

Found 8 of 8 tickers in ../data/datasets/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in ../data/datasets/daily_benchmark_quotes.csv
Missing tickers: []

Sharpe Ratios for individual assets:
AMAT: 0.5155
DE: 0.8804
GILD: 0.4915
HD: 0.4365
PG: 0.5571
PGR: 1.0935
TMUS: 0.7528
XOM: 0.6783


,AMAT,DE,GILD,HD,PG,PGR,TMUS,XOM
Date,,,,,,,,
2020-06-02,0.001871,0.025192,-0.025727,0.021697,0.006825,0.009408,0.007052,0.021991
2020-06-03,0.035620,0.034682,0.015581,-0.000848,0.004016,0.000419,0.007912,0.039782
2020-06-04,0.005397,0.017464,0.041243,-0.008161,-0.021118,-0.003919,-0.010258,-0.002808
2020-06-05,0.034906,0.034039,-0.010152,0.023584,0.019399,0.038249,0.009550,0.077917
2020-06-08,-0.004863,0.012487,0.002866,0.007317,0.006099,-0.004465,0.032333,0.030965


### Step 5: Plot Return Comparisons

In [8]:
# Calculate portfolio returns using weights from Excel file
portfolio_weights = portfolio_df.set_index('Ticker')['Weight'].to_dict()
weighted_returns = pd.DataFrame()

print("\nPortfolio Weights:")
for ticker in portfolio_tickers:
    weight = portfolio_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_returns.columns:
        weighted_returns[ticker] = stock_returns[ticker] * weight

# Sum across all weighted returns to get the portfolio return
portfolio_return = weighted_returns.sum(axis=1)

# Create equal-weight portfolio for comparison
equal_weight = 1/len([t for t in portfolio_tickers if t in stock_returns.columns])
equal_weighted_returns = pd.DataFrame()

for ticker in portfolio_tickers:
    if ticker in stock_returns.columns:
        equal_weighted_returns[ticker] = stock_returns[ticker] * equal_weight
        
equal_weight_return = equal_weighted_returns.sum(axis=1)

print("\nPortfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_return):.4f}")

plt.figure(figsize=(12, 6))
(1 + portfolio_return).cumprod().plot(label='Weighted Portfolio')
(1 + equal_weight_return).cumprod().plot(label='Equal Weight')
(1 + benchmark_returns).cumprod().plot(label=benchmark)
plt.legend()
plt.title('Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()


Portfolio Weights:
PGR: 40.00%
DE: 21.22%
TMUS: 11.75%
XOM: 7.03%
HD: 5.00%
AMAT: 5.00%
PG: 5.00%
GILD: 5.00%

Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 1.3159
Sharpe Ratio (Equal-Weight): 1.1418


### Step 6: Generate Reports

#### Portfolio (Weighted) vs Benchmark

In [9]:
portfolio_return = portfolio_return.resample('D').sum()  # Example of valid aggregation
portfolio_return.name = "Weighted Portfolio"

# Generate Report
qs.reports.html(
    portfolio_return,
    benchmark_returns,
    rf=risk_free_rate,
    figsize=(8, 5),
    # output=f'portfolio_vs_{benchmark}.html',
    output=f'portfolio_vs_benchmark-{datetime.date(end_date)}.html',
    title=f'Portfolio vs {benchmark} (Benchmark)',
    benchmark_title=f'{benchmark}',
    download_filename=f'portfolio_vs_{benchmark}.html'
)

qs.reports.full(
    portfolio_return, 
    benchmark_returns,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title=f'Portfolio vs {benchmark}',
    benchmark_title=f'{benchmark}') 

/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To

Added download button and removed QuantStats attribution from portfolio_vs_benchmark-2025-05-28.html


/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To

                           DLN         Strategy
-------------------------  ----------  ----------
Start Period               2020-06-03  2020-06-03
End Period                 2025-05-28  2025-05-28
Risk-Free Rate             4.22%       4.22%
Time in Market             69.0%       69.0%

Cumulative Return          79.4%       184.26%
CAGR﹪                     8.43%       15.56%

Sharpe                     0.39        0.79
Prob. Sharpe Ratio         56.33%      85.97%
Smart Sharpe               0.39        0.79
Sortino                    0.54        1.13
Smart Sortino              0.54        1.13
Sortino/√2                 0.38        0.8
Smart Sortino/√2           0.38        0.8
Omega                      1.18        1.18

Max Drawdown               -17.03%     -12.32%
Longest DD Days            601         241
Volatility (ann.)          12.07%      14.34%
R^2                        0.39        0.39
Information Ratio          0.04        0.04
Calmar                     0.5         1.

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2022-04-11,2022-06-23,2022-08-16,128,-12.321047,-12.252188
2,2021-05-10,2021-10-12,2022-01-05,241,-11.041445,-10.476656
3,2025-03-18,2025-04-08,2025-05-18,62,-10.863782,-10.585661
4,2022-08-26,2022-09-30,2022-10-27,63,-10.688655,-10.122853
5,2024-12-02,2025-01-10,2025-02-12,73,-10.574161,-9.825624


/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/home/renanmogo/

#### Portfolio (Weighted) vs Portolio (Equally-Weighted)

In [10]:
# Ensure equal_weight_return and portfolio_return are properly aggregated if resampled
equal_weight_return = equal_weight_return.resample('D').sum() if equal_weight_return.index.freq is None else equal_weight_return
portfolio_return = portfolio_return.resample('D').sum() if portfolio_return.index.freq is None else portfolio_return

# Set the name for the equal weight portfolio
equal_weight_return.name = "Equal Weight Portfolio"

# Generate Report
qs.reports.html(
    portfolio_return,
    equal_weight_return,
    rf=risk_free_rate,
    figsize=(8, 5),
    output=f'portfolio_vs_equal_weight-{datetime.date(end_date)}.html',
    title='Portfolio (Weighted) vs Portolio (Equally-Weighted)',
    benchmark_title="Equal Weight Portfolio",
    download_filename="portfolio_vs_equal_weight.html" 
)

qs.reports.full(
    portfolio_return, 
    equal_weight_return,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title='Portfolio vs Equal Weight',
    benchmark_title="Equal Weight Portfolio",
    ) 

/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To

Added download button and removed QuantStats attribution from portfolio_vs_equal_weight-2025-05-28.html


/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To

                           Equal Weight Portfolio    Strategy
-------------------------  ------------------------  ----------
Start Period               2020-06-02                2020-06-02
End Period                 2025-05-28                2025-05-28
Risk-Free Rate             4.22%                     4.22%
Time in Market             69.0%                     69.0%

Cumulative Return          132.32%                   187.59%
CAGR﹪                     12.37%                    15.74%

Sharpe                     0.63                      0.8
Prob. Sharpe Ratio         76.85%                    86.59%
Smart Sharpe               0.63                      0.8
Sortino                    0.91                      1.15
Smart Sortino              0.91                      1.15
Sortino/√2                 0.64                      0.81
Smart Sortino/√2           0.64                      0.81
Omega                      1.18                      1.18

Max Drawdown               -13.49%       

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2022-04-11,2022-06-23,2022-08-16,128,-12.321047,-12.252188
2,2021-05-10,2021-10-12,2022-01-05,241,-11.041445,-10.476656
3,2025-03-18,2025-04-08,2025-05-18,62,-10.863782,-10.585661
4,2022-08-26,2022-09-30,2022-10-27,63,-10.688655,-10.122853
5,2024-12-02,2025-01-10,2025-02-12,73,-10.574161,-9.825624


/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/home/renanmogo/mfin-portfolio-management/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/home/renanmogo/